# HASIL PENELITIAN
> Dengan menggunakan seluruh atribut (24 atribut prediktif),
tanpa penanganan outlier (400 data), nilai k terbaik pada kNN = 3, dengan akurasi 99.25%

> Dengan menggunakan backward elimination, terpilih 10 atribut: sg, al, bu, sc, sod, hemo, rbc, htn, dm, appet
tanpa penanganan outlier (400 data), nilai k terbaik pada kNN = 3, dengan akurasi 99.25%

> Dengan 10 atribut yang terpilih, meskipun akurasi tidak menunjukkan peningkatan, namun proses komputasi menjadi lebih cepat

# I. PEMAHAMAN DOMAIN DAN TUJUAN

## 1.1. Pemahaman Penyakit Ginjal Kronis

>Penyakit Ginjal Kronis (PGK) adalah suatu proses patofisiologis dengan etiologi yang beragam, mengakibatkan penurunan fungsi ginjal yang progresif, penurunan fungsi ini bersifat kronis dan irreversible.

>Data didapatkan dari UCI Machine Learning Repository. Berisikan 400 data dan memiliki 25 atribut, terdiri dari 1 kelas target dan 24 atribut. 

## 1.2. Tujuan

>Identifikasi PGK dengan atribut yang paling berpengaruh

# II. PEMBUATAN DATASET DAN TARGET

## 2.1 Data UCI

Data didapat dari: https://archive.ics.uci.edu/ml/datasets/chronic_kidney_disease

Data tersebut di-import dalam mariaDB

In [1]:
import numpy as np
import pandas as pd
import pymysql as pskl
import matplotlib.pyplot as plt

# Menampilkan semua array
# np.set_printoptions(threshold=np.nan)

In [2]:
def dataset(retrieve="all", id=1, target=1): #hasil bertipe dataframe
    """
    parameters
    ----------
    retrieve: (all, numeric, polinom)
        all    : semua kolom
        numeric: hanya kolom numeric
        polinom: hanya kolom polinom
    id: (0, 1)
        0: tanpa kolom id
        1: dengan kolom id
    class: (0, 1)
        0: tanpa kolom class
        1: dengan kolom class
    
    """
    connection = pskl.connect(host="localhost", user="root", passwd="", database="knn")
    cursor = connection.cursor()
    if(retrieve=="all"):
        cols = ["id", "age", "bp", "sg", "al", "su", "bgr", "bu", "sc", "sod", "pot", "hemo", "pcv", "wbcc", "rbcc", "rbc", "pc", "pcc", "ba", "htn", "dm", "cad", "appet", "pe", "ane", "class"]
        retrieve = "SELECT id, age, bp, sg, al, su, bgr, bu, sc, sod, pot, hemo, pcv, wbcc, rbcc, rbc, pc, pcc, ba, htn, dm, cad, appet, pe, ane, class FROM ckd_preprocessing3"
    elif(retrieve=="numeric"):
        cols = ["id", "age", "bp", "sg", "al", "su", "bgr", "bu", "sc", "sod", "pot", "hemo", "pcv", "wbcc", "rbcc", "class"]
        retrieve = "SELECT id, age, bp, sg, al, su, bgr, bu, sc, sod, pot, hemo, pcv, wbcc, rbcc, class FROM ckd_preprocessing3"
    elif(retrieve=="polinom"):
        cols = ["id", "rbc", "pc", "pcc", "ba", "htn", "dm", "cad", "appet", "pe", "ane", "class"]
        retrieve = "SELECT id, rbc, pc, pcc, ba, htn, dm, cad, appet, pe, ane, class FROM ckd_preprocessing3"
    resolveall = cursor.execute(retrieve)
    rows_tupple = cursor.fetchall()
    data = pd.DataFrame(list(rows_tupple))
    data.columns = cols
    if(id==0):
        data = data.drop(["id"], axis=1)
    if(target==0):
        data = data.drop(["class"], axis=1)
    data = data.fillna(value=np.nan) #mengubah missing value menjadi NaN
    return data

In [3]:
dataset_df = dataset()
print(dataset_df.shape)
dataset_df.head()

(400, 26)


,id,age,bp,sg,al,su,bgr,bu,sc,sod,...,pc,pcc,ba,htn,dm,cad,appet,pe,ane,class
0,1,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,NaN,...,normal,notpresent,notpresent,yes,yes,no,good,no,no,ckd
1,2,7.0,50.0,1.020,4.0,0.0,NaN,18.0,0.8,NaN,...,normal,notpresent,notpresent,no,no,no,good,no,no,ckd
2,3,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,NaN,...,normal,notpresent,notpresent,no,yes,no,poor,no,yes,ckd
3,4,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,...,abnormal,present,notpresent,yes,no,no,poor,yes,yes,ckd
4,5,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,NaN,...,normal,notpresent,notpresent,no,no,no,good,no,no,ckd


## 2.2. Menentukan Atribut Target

>Atribut "class" sebagai target dataset

outputnya target

# III. DATA CLEANING DAN PREPROCESSING

## 3.1 Menghilangkan Outliers

>referensi: http://digilib.unila.ac.id/20585/4/II.%20TINJAUAN%20PUSTAKA.pdf

# IV. TRANSFORMASI DATA

## 4.1 Data Nominal Menjadi Numeric

>Mengubah atribut / kolom polinom menjadi numerik

>Dikarenakan semua kolom nominal hanya memiliki 2 kelas (binary), hasil transformasi menjadi 0 dan 1

In [4]:
def encoding(df, column="all_nominal"):
    """
    paramters:
    ----------
    df: input tipe dataframe
    column: input tipe string, menentukan kolom nominal mana yang akan di transformasi menjadi numerik
    
    output:
    ----------
    mengembalikan dataframe dengan kolom tertentu yang sudah di encode selain "NaN"
    
    """
    #python melakukan pass by reference, sehingga dibuat copy agar df sebelumnya tidak berubah
    
    copy_df = df.copy()
    if(column!="all_nominal"):
        a = copy_df[column].unique().tolist()
        c = [x for x in a if str(x) != 'nan']
        l=[]
        i=0
        for x in c:
            l.append(i)
            i+=1
        copy_df[column] = copy_df[column].replace(c, l)
    
    elif(column=="all_nominal"):
        all_nominal = ["rbc", "pc", "pcc", "ba", "htn", "dm", "cad", "appet", "pe", "ane","class"]
        for col in all_nominal:
            copy_df = encoding(copy_df, col) #rekursif
            
    return copy_df

In [5]:
encoded_df = encoding(dataset_df)
print(encoded_df.shape)
encoded_df.head()

(400, 26)


,id,age,bp,sg,al,su,bgr,bu,sc,sod,...,pc,pcc,ba,htn,dm,cad,appet,pe,ane,class
0,1,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,7.0,50.0,1.020,4.0,0.0,NaN,18.0,0.8,NaN,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0
2,3,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0
3,4,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0
4,5,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,NaN,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0


## 4.2 Penanganan Data yang Hilang

Mengubah NaN menjadi strategi tertentu, dalam hal ini diubah menjadi rata - rata dari setiap kolom

In [6]:
from sklearn.preprocessing import Imputer

In [7]:
def missing_handling(df, column="all", method="mean"):
    """
    paramters:
    ----------
    df: input tipe dataframe
    column: menentukan kolom numerik mana yang akan dilakukan penanganan missing value
    method: strategi penanganan missing value
    
    output:
    ----------
    mengembalikan dataframe dengan kolom yang sudah dilakukan penanganan NaN
    """
    
    copy_df = df
    imputer = Imputer(missing_values="NaN", strategy=method, axis = 0)    
    
    if(column!="all"):
        imputer = imputer.fit(copy_df[[column]])
        filledmissing_df = imputer.transform(copy_df[[column]])
        df_change = filledmissing_df.ravel()    
        copy_df[column] = df_change
        
    elif(column=="all"):
        all_col = ["age", "bp", "sg", "al", "su", "bgr", "bu", "sc", "sod", "pot", "hemo", "pcv", "wbcc", "rbcc", "rbc", "pc", "pcc", "ba", "htn", "dm", "cad", "appet", "pe", "ane", "class"]
        for col in all_col:
            copy_df = missing_handling(copy_df, col, method) #rekursif
    return copy_df

In [8]:
missing_handling_df = missing_handling(encoded_df, "all")
print(missing_handling_df.shape)
missing_handling_df.head()

(400, 26)


,id,age,bp,sg,al,su,bgr,bu,sc,sod,...,pc,pcc,ba,htn,dm,cad,appet,pe,ane,class
0,1,48.0,80.0,1.020,1.0,0.0,121.000000,36.0,1.2,137.528754,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,7.0,50.0,1.020,4.0,0.0,148.036517,18.0,0.8,137.528754,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,62.0,80.0,1.010,2.0,3.0,423.000000,53.0,1.8,137.528754,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,4,48.0,70.0,1.005,4.0,0.0,117.000000,56.0,3.8,111.000000,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
4,5,51.0,80.0,1.010,2.0,0.0,106.000000,26.0,1.4,137.528754,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


## 4.3 Normalisasi

>Melakukan normalisasi data, agar range tiap atribut sama dan tidak ada yang dominan saat perhitungan

$$Xnorm = \frac{X - min(X)}{max(X)-min(X)}$$

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
def normalizing(df, column="all", f_range=(0,1)):
    """
    paramters:
    ----------
    df: input tipe dataframe
    column: menentukan kolom numerik mana yang akan dilakukan penanganan missing value
    range: range normalisasi
    
    output:
    ----------
    mengembalikan dataframe dengan kolom yang sudah dinormalisasi
    """
    
    copy_df = df.copy()
    scale = MinMaxScaler(feature_range=f_range)    
    
    if(column!="all"):
        normalization_array = scale.fit_transform(copy_df[[column]])
        df_change = normalization_array.ravel()    
        copy_df[column] = df_change
        
    elif(column=="all"):
        all_col = ["age", "bp", "sg", "al", "su", "bgr", "bu", "sc", "sod", "pot", "hemo", "pcv", "wbcc", "rbcc", "rbc", "pc", "pcc", "ba", "htn", "dm", "cad", "appet", "pe", "ane"]
        for col in all_col:
            copy_df = normalizing(copy_df, col, f_range) #rekursif
    return copy_df

In [11]:
normalized_df = normalizing(missing_handling_df, column="all", f_range=(0,1))
print(normalized_df.shape)
normalized_df.head()

(400, 26)


,id,age,bp,sg,al,su,bgr,bu,sc,sod,...,pc,pcc,ba,htn,dm,cad,appet,pe,ane,class
0,1,0.522727,0.230769,0.75,0.2,0.0,0.211538,0.088575,0.010582,0.839298,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.056818,0.000000,0.75,0.8,0.0,0.269309,0.042362,0.005291,0.839298,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,0.681818,0.230769,0.25,0.4,0.6,0.856838,0.132221,0.018519,0.839298,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,4,0.522727,0.153846,0.00,0.8,0.0,0.202991,0.139923,0.044974,0.671924,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
4,5,0.556818,0.230769,0.25,0.4,0.0,0.179487,0.062901,0.013228,0.839298,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# path=r"C:\Users\Ikhsan\Desktop"
# import os
# normalized_df.to_csv(os.path.join(path,r'preprocessing.csv'))

# V. PENENTUAN TUGAS DATA MINING

>Tugas Data Mining yang dilakukan adalah klasifikasi

# VI. PENENTUAN ALGORITMA DAN METODE

## 6.1 Klasifikasi dengan kNN

In [13]:
import MyDir as my

## 6.2 Feature Selection dengan Backward Elimination

# VII. Implementasi Algoritma Data Mining

## 7.1 Mencari Nilai k Terbaik pada kNN

In [14]:
from sklearn.model_selection import KFold
def knn_kfold(df, kNN = 5, k_fold=10, shuffle_=True, arr_akurasi=0):
    """
    Parameter:
    ----------
    df : Dataframe, data bertipe float / integer, tidak boleh ada NaN
    kNN : Nilai k-tetangga/data terdekat, default = 5
    k_fold : Jumlah fold / lipatan split training - test. Default = 5
    shuffle_ : Bentuk shuffle split jika diperlukan. Default = True

    Ouput:
    -------
    arr_akurasi : 1 untuk menghasilkan output array akurasi tiap fold, 0 untuk rata - rata akurasi. default = 0
    """
    
    kf = KFold(n_splits = k_fold, shuffle = shuffle_, random_state=2)
    akurasi = []
    for train_index, test_index in kf.split(df):
        train_df = df.iloc[train_index]
        x_train = np.array(train_df.iloc[:,1:-1])
        y_train = np.array(train_df["class"])
        test_df = df.iloc[test_index]
        x_test = np.array(test_df.iloc[:,1:-1])
        y_test = np.array(test_df["class"])
    
        my_predictions = np.array([my.knn_predict(p, x_train, y_train, k=kNN) for p in x_test])
        akurasi_my_predictions = np.mean(my_predictions == y_test)*100
        akurasi.append(akurasi_my_predictions)
    if(arr_akurasi==1):
        return akurasi
    else:
        return np.mean(akurasi)

>Dikarenakan data target sudah terurut dari 0"notckd" terlebih dahulu kemudian 1"ckd", KFold dengan metode pengambilan data training dan data test secara random (shuffle=True) lebih baik, karena data training dan test terdistribusi lebih seimbang.

In [15]:
def find_best_knn(df, start=3, finish=16, step=2, graph=True):
    akurasi = []
    maks = 0
    for i in range (start, finish, step):
        akurasi_k = knn_kfold(df, kNN = i, k_fold=10, shuffle_=True, arr_akurasi=0)
        akurasi.append([i, akurasi_k])
        if(maks<akurasi_k):
            maks =akurasi_k
            k = i
            print(k, maks)
        if (graph==True):
            x = [i[0] for i in akurasi]
            y = [i[1] for i in akurasi]
            plt.plot(x,y)
            plt.title("Mencari k Terbaik kNN")
            plt.xlabel("k")
            plt.ylabel("akurasi")
            plt.grid()
            plt.show
    return akurasi

In [16]:
k_knn = find_best_knn(normalized_df)
k_knn

3 99.25


[[3, 99.25],
 [5, 99.0],
 [7, 98.25],
 [9, 98.0],
 [11, 97.75],
 [13, 97.75],
 [15, 97.75]]

## 7.2 Backward Elimination

In [17]:
import statsmodels.formula.api as sm

def backward_elimination(splitdf_x, splitdf_y, sl=0.05):
    sl_ = sl
    copy_df = splitdf_x.copy()
    reg_ols = sm.OLS(endog=splitdf_y, exog=copy_df).fit()
    pval = reg_ols.pvalues
    maks_pval = max(pval)
    
    if(maks_pval >= sl_):
        delete_col = pval[pval==maks_pval].index[0]
        drp = [delete_col]
        copy_df = copy_df.drop(drp, axis=1)
        copy_df = backward_elimination(copy_df, splitdf_y, sl_)

    else:
        copy_df = pd.concat([copy_df, splitdf_y], axis=1)
    return copy_df

In [18]:
x = normalized_df.iloc[:,1:-1]
print(x.shape)
x.head()

(400, 24)


,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,rbc,pc,pcc,ba,htn,dm,cad,appet,pe,ane
0,0.522727,0.230769,0.75,0.2,0.0,0.211538,0.088575,0.010582,0.839298,0.047803,...,0.189516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.056818,0.000000,0.75,0.8,0.0,0.269309,0.042362,0.005291,0.839298,0.047803,...,0.189516,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.681818,0.230769,0.25,0.4,0.6,0.856838,0.132221,0.018519,0.839298,0.047803,...,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,0.522727,0.153846,0.00,0.8,0.0,0.202991,0.139923,0.044974,0.671924,0.000000,...,0.000000,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
4,0.556818,0.230769,0.25,0.4,0.0,0.179487,0.062901,0.013228,0.839298,0.047803,...,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [19]:
y = normalized_df.iloc[:,25:]
print(y.shape)
y.head()

(400, 1)


,class
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [28]:
backwarded_df = backward_elimination(x,y,0.05)
print(backwarded_df.shape)
backwarded_df.head()

(400, 11)


,sg,al,bu,sc,sod,hemo,rbc,htn,dm,appet,class
0,0.75,0.2,0.088575,0.010582,0.839298,0.836735,0.189516,0.0,0.0,0.0,0.0
1,0.75,0.8,0.042362,0.005291,0.839298,0.557823,0.189516,1.0,1.0,0.0,0.0
2,0.25,0.4,0.132221,0.018519,0.839298,0.442177,0.000000,1.0,0.0,1.0,0.0
3,0.00,0.8,0.139923,0.044974,0.671924,0.551020,0.000000,0.0,1.0,1.0,0.0
4,0.25,0.4,0.062901,0.013228,0.839298,0.578231,0.000000,1.0,1.0,0.0,0.0


In [29]:
print(backwarded_df.shape)
backwarded_k_knn = find_best_knn(backwarded_df)
backwarded_k_knn

(400, 11)
3 99.25


[[3, 99.25],
 [5, 99.25],
 [7, 99.0],
 [9, 99.0],
 [11, 99.0],
 [13, 99.0],
 [15, 99.0]]

In [31]:
backwarded_knn = knn_kfold(backwarded_df, kNN = 5, k_fold=10, shuffle_=True, arr_akurasi=1)
backwarded_knn

[100.0, 97.5, 97.5, 100.0, 97.5, 100.0, 100.0, 100.0, 100.0, 100.0]